## Create parameters file

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

from hscpy import mitchell, realisation
from futils import parse_version

NCELLS = 100_000
REPLICATES = 100
PATH2HSC = Path("~").expanduser() / "hsc"

In [ ]:
%%bash -s "$PATH2HSC" --out version
$1/target/release/hsc  --version

In [ ]:
VERSION = parse_version(version)
PATH2SAVE = Path(f"./{VERSION}")

print("Running hsc with version:", VERSION)

In [ ]:
donors = mitchell.donors()
donors

In [ ]:
def tau_exp():
    # 0.75 is 9/12 years for the newborns
     return 0.75/np.log(NCELLS)
    
    
def create_parameters(
    donors: pd.DataFrame, name: str, eta: float, sigma: float, mu: int, tau: float
) -> realisation.SimulationCMD:
    return realisation.SimulationCMD(
        cells=NCELLS,
        sample=donors[donors.name == name].cells.squeeze(),
        eta=eta,
        sigma=sigma,
        mu=mu,
        tau=tau,
        age=donors[donors.name == name].age.squeeze(),
        name=name,
        seed=None,
        tau_exp=tau_exp(),
        dir2save=(Path(".").expanduser() / VERSION).absolute(),
    )

In [ ]:
# params are from ABC
parameters = dict()

name = "CB001"
eta, sigma, mu, tau = 0.03, 0.01, 1, 1
parameters[name] = create_parameters(donors, name, eta, sigma, mu, tau)

# assume same params for both 0 age donors
name = "CB002"
eta, sigma, mu, tau = 0.03, 0.01, 1, 1
parameters[name] = create_parameters(donors, name, eta, sigma, mu, tau)


name = "KX001"
eta, sigma, mu, tau = 0.03, 0.01, 1, 1.8
parameters[name] = create_parameters(donors, name, eta, sigma, mu, tau)


name = "KX002"
eta, sigma, mu, tau = 0.02, 0.01, 1, 2.6
parameters[name] = create_parameters(donors, name, eta, sigma, mu, tau)


name = "SX001"
eta, sigma, mu, tau = 0.03, 0.01, 1, 2.6
parameters[name] = create_parameters(donors, name, eta, sigma, mu, tau)


name = "AX001"
eta, sigma, mu, tau = 0.02, 0.01, 1, 3
parameters[name] = create_parameters(donors, name, eta, sigma, mu, tau)


name = "KX008"
eta, sigma, mu, tau = 0.06, 0.04, 16, 5
parameters[name] = create_parameters(donors, name, eta, sigma, mu, tau)


name = "KX004"
eta, sigma, mu, tau = 0.12, 0.04, 16, 1.4
parameters[name] = create_parameters(donors, name, eta, sigma, mu, tau)


name = "KX003"
eta, sigma, mu, tau = 0.08, 0.04, 23, 3
parameters[name] = create_parameters(donors, name, eta, sigma, mu, tau)

In [ ]:
with open("parameters.txt", "w") as f:
    for params in parameters.values():
        for _ in range(REPLICATES):
            f.write(params.parameters() + "\n")

In [ ]:
%%bash
head parameters.txt

In [ ]:
%%bash
tail parameters.txt

In [ ]:
%%bash
wc -l parameters.txt

In [ ]:
%%bash --out runs
wc -l parameters.txt

In [ ]:
runs = int(runs.split(" ")[0])
assert REPLICATES * len(parameters) == runs, f"{REPLICATES * len(parameters)} vs {runs}"

In [ ]:
REPLICATES * len(parameters)